In [ ]:
# Libraries
import pandas as pd
import numpy as np
import sympy as sp
import scipy.optimize as opt

In [ ]:
# Parameters
v = 4
t = 1
c_1 = 1
c_2 = 0
a_1 = 1.5
a_2 = 2.5

# Demand function
def demand(v, p_1, p_2, t, a_1, a_2):
    d_1 = (2 * v + p_2 - 3 * p_1 + t * (a_2 - a_1)) / (2 * t)
    d_2 = (2 * v + p_1 - 3 * p_2 + t * (a_2 - a_1)) / (2 * t)
    return np.array([d_1, d_2])

# Indifferent consumers' address function
def indiff_consumers_address(p_1, p_2, a_1, a_2, t):
    x_0 = (a_1 + a_2) / 2 - (p_1 - p_2) / 2
    x_1 = a_1 - (v - p_1) / t
    x_2 = a_2 + (v - p_2) / t
    return np.array([x_0, x_1, x_2])

# Wholesale price when firms are separate
def wholesale_price_sep(v, t, c_1, c_2):
    return (v / 2) + ((t - c_1 - c_2) / 4)

# Retail price when firms are separate
def retail_price_sep(v, t, c_1, c_2, w):
    p_1 = (14 * v + 7 * t + 21 * w + 18 * c_1 + 3 * c_2) / 35
    p_2 = (14 * v + 7 * t + 21 * w + 18 * c_2 + 3 * c_1) / 35
    return np.array([p_1, p_2])

# Profit function for Firm 01 (upstream firm)
def pi_01(w, p_1):
    p_2 = (2 * v + t + 3 * w + 3 * c_2 + p_1) / 6
    d = demand(v, p_1, p_2, t, a_1, a_2)
    d_1, d_2 = d[0], d[1]
    profit = (1 / (2 * t)) * (
        w * (2 * v + p_1 - 3 * p_2 + t) +
        (p_1 - c_1) * (2 * v + p_2 - 3 * p_1 + t)
    )
    return -profit  # Minimize the negative profit to maximize it

# Profit function for Firm 2 (downstream firm)
def pi_2(w, p_1, p_2):
    d = demand(v, p_1, p_2, t, a_1, a_2)
    d_2 = d[1]
    profit = d_2 * (p_2 - w - c_2)
    return -profit  # Minimize the negative profit to maximize it

# Objective function for Firm 01's optimization (maximize profit with respect to w and p_1)
def objective_firm_01(x):
    w, p_1 = x
    return pi_01(w, p_1)

# Objective function for Firm 2's optimization (maximize profit with respect to p_2)
def objective_firm_2(x, w, p_1):
    p_2 = x
    return pi_2(w, p_1, p_2)

# Solve the system of optimization problems
def solve_optimal_prices():
    # Initial guesses for w and p_1 (for Firm 01)
    initial_guess_01 = [3, 3]

    # Solve for w and p_1 using minimize
    result_01 = opt.minimize(objective_firm_01, initial_guess_01, bounds=[(0, 5), (0, 5)])

    # Extract the optimal values for w and p_1
    w_opt, p_1_opt = result_01.x

    # Now, solve for the optimal p_2 using minimize (Firm 2's optimization)
    result_2 = opt.minimize(objective_firm_2, [2], args=(w_opt, p_1_opt), bounds=[(0, 5)])

    # Extract the optimal value for p_2
    p_2_opt = result_2.x[0]

    return w_opt, p_1_opt, p_2_opt

# Consumer surplus as a sum of integrals over different locations bands
def consumer_surplus(v, p1, p2, a1, a2, x0, x1, x2, t):
    x = sp.Symbol('x')

    F1 = v*x - p1*x - a1*x*t + (x**2)*t/2 # left of a_1, right of x_1
    F2 = v*x - p1*x + a1*x*t - (x**2)*t/2 # left of x_0, right of a_1
    F3 = v*x - p2*x - a2*x*t + (x**2)*t/2 # left of a_2, right of x_0
    F4 = v*x - p2*x + a2*x*t - (x**2)*t/2 # left of x_2, right of a_2

    term1 = F1.subs(x, a1) - F1.subs(x, x1)
    term2 = F2.subs(x, x0) - F2.subs(x, a1)
    term3 = F3.subs(x, a2) - F3.subs(x, x0)
    term4 = F4.subs(x, x2) - F4.subs(x, a2)

    result = term1 + term2 + term3 + term4
    return result.simplify()

def main():
    # Separated Firms Optimization
    w_sep = wholesale_price_sep(v, t, c_1, c_2)
    p_sep = retail_price_sep(v, t, c_1, c_2, w_sep)
    x_sep = indiff_consumers_address(p_sep[0], p_sep[1], a_1, a_2, t)
    consumer_surplus_sep = consumer_surplus(v, p_sep[0], p_sep[1], a_1, a_2, x_sep[0], x_sep[1], x_sep[2], t)
    pi_01_sep = pi_01(w_sep, p_sep[0])
    pi_2_sep = pi_2(w_sep, p_sep[0], p_sep[1])

    # Integrated Firms Optimization
    w_opt, p_1_opt, p_2_opt = solve_optimal_prices()
    x_opt = indiff_consumers_address(p_1_opt, p_2_opt, a_1, a_2, t)
    consumer_surplus_opt = consumer_surplus(v, p_1_opt, p_2_opt, a_1, a_2, x_opt[0], x_opt[1], x_opt[2], t)
    pi_01_opt = pi_01(w_opt, p_1_opt)
    pi_2_opt = pi_2(w_opt, p_1_opt, p_2_opt)

    # Create a DataFrame to store the values
    data = {
        'Indicator': ['x_0', 'x_1', 'x_2', 'w', 'p_1', 'p_2', 'pi_01', 'pi_2', 'Consumer Surplus'],
        'Separation': [x_sep[0], x_sep[1], x_sep[2],
                       w_sep, p_sep[0], p_sep[1],
                       -pi_01_sep, -pi_2_sep, consumer_surplus_sep],
        'Integration': [x_opt[0], x_opt[1], x_opt[2],
                        w_opt, p_1_opt, p_2_opt,
                        -pi_01_opt, -pi_2_opt, consumer_surplus_opt]
    }

    df = pd.DataFrame(data).set_index('Indicator').astype(float).round(3)
    return df

df = main()
df

,Separation,Integration
Indicator,,
x_0,1.786,2.167
x_1,1.014,0.250
x_2,3.414,3.417
w,2.000,2.250
p_1,3.514,2.750
p_2,3.086,3.083
pi_01,5.197,6.167
pi_2,1.768,1.042
Consumer Surplus,1.032,2.063
